In [3]:
# Bringing in the data

import pandas as pd

# Read the CSV file into a DataFrame
df_mm = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/mm_data.csv")

df_p = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/p_data.csv")

In [85]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor

def multi_logistic_accuracy(df, model, selected_feature_tuple):
    """
    Custom cross-validation for predicting NBA champion using Logistic Regression.
    
    Parameters:
    - df: DataFrame containing the data.
    - model: Logistic Regression model.
    - selected_feature_tuple: Tuple of selected features.
    
    Returns:
    - Accuracy DataFrame sorted by accuracy.
    """
    selected_features = list(selected_feature_tuple)  # Convert tuple to list
    results = []

    for test_year in range(2014, 2024):
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        test_data = df[df['Year'] == test_year]

        X_train, y_train = train_data[selected_features], train_data['Levels']
        X_test, y_test = test_data[selected_features], test_data['Levels']

        # Print VIF if multiple features
        #if len(selected_features) > 1:
            #vif_data = pd.DataFrame({
                #"Feature": selected_features,
                #"VIF": [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
            #})
            #print(f"VIF for {selected_features}:\n{vif_data}\n")

        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted scores
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 4]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        predicted_champion = ranked_teams.iloc[0]['Team']
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Convert boolean comparison to integer (1 if correct, 0 if incorrect)
        accuracy = int(predicted_champion == actual_champion)
        results.append(accuracy)  # Store for each year

    # Compute final accuracy as a percentage
    overall_accuracy = sum(results) / len(results)

    accuracies = pd.DataFrame({'Selected Features': [selected_feature_tuple], 'Accuracy': [overall_accuracy]})
    
    # Sort by Accuracy in descending order (higher accuracy first)
    accuracies_sorted = accuracies.sort_values(by='Accuracy', ascending=False)

    return accuracies_sorted

In [45]:
# defining the cross validation technique for our use case

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor

def multi_logistic_top5(df, model, selected_features):
    """
    Custom cross-validation for predicting NBA champion using XGBoostRanker,
    with group information to rank the teams by their predicted level (score).
    """
    results = []

    # Iterate over the years you want to predict (2014 to 2023)
    for test_year in range(2014, 2024):
        
        # Training data: the previous 5 years of data (e.g., 2009-2013 for 2014 prediction)
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        
        # Test data: current year
        test_data = df[df['Year'] == test_year]
        
        # Prepare features and target for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['Levels']
        
        X_test = test_data[selected_features]
        y_test = test_data['Levels']

        # If more than 1 feature is selected, calculate VIF
        if len(selected_features) > 1:
            vif_data = pd.DataFrame()
            vif_data["Feature"] = X_train.columns
            vif_data["VIF"] = [variance_inflation_factor(X_train.values, i) for i in range(X_train.shape[1])]
            print(vif_data)
        
        # Train the model using Logistic Regression
        model.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted scores
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 4]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Get top 5 teams and their scores
        top_teams = ranked_teams.head(5)
        top_teams_list = top_teams.apply(lambda row: f"{row['Team']}: {row['Score']:.4f}", axis=1).tolist()
        
        # Store results
        results.append([test_year, actual_champion] + top_teams_list)
    
    # Create dataframe with results
    columns = ['Year', 'Actual Champion', 'First Pick', 'Second Pick', 'Third Pick', 'Fourth Pick', 'Fifth Pick']
    results_df = pd.DataFrame(results, columns=columns)
    results_df = results_df.sort_values(by="Year", ascending=False)

    # Compute accuracy
    accuracy = (results_df['Actual Champion'] == results_df['First Pick'].str.split(': ').str[0]).mean()
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return results_df

In [131]:
# min max feature importance

from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

champion_mm = df_mm[df_mm['Champion'] == 1]
champion_mm = champion_mm.drop(columns=['L', 'PL', 'Champion', 'Levels','Year', 'Team'])

col_means = champion_mm.mean()
sorted_col_means = col_means.sort_values(ascending=False)
print(sorted_col_means.head(10))

col_meds = champion_mm.median()
sorted_col_meds = col_meds.sort_values(ascending=False)
print(sorted_col_meds.head(10))

col_mins = champion_mm.min()
sorted_col_mins = col_mins.sort_values(ascending=False)
print(sorted_col_mins.head(10))

important_mm = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

print(champion_mm[important_mm])

pairs1_mm = list(combinations(important_mm, 1))
pairs2_mm = list(combinations(important_mm, 2))
pairs3_mm = list(combinations(important_mm, 3))
pairs4_mm = list(combinations(important_mm, 4))
pairs5_mm = list(combinations(important_mm, 5))

for pair in pairs2_mm:
    vif_data = pd.DataFrame()
    df_subset = df_mm[list(pair)]  # Select only the columns in the pair

    vif_data["Feature"] = df_subset.columns
    vif_data["VIF"] = [variance_inflation_factor(df_subset.values, i) for i in range(df_subset.shape[1])]

    print(f"VIF for features: {pair}")
    print(vif_data)
    print("-" * 50)  # Separator for better readability

W            0.907581
PW           0.891975
SRS          0.887981
NRtg         0.883819
MOV          0.882850
eFG%         0.860737
FG%          0.836710
2P%          0.828092
TS%          0.827180
0-3ft_FG%    0.825777
dtype: float64
eFG%         0.935484
W            0.906977
PW           0.880952
MOV          0.859944
NRtg         0.859459
FG%          0.859375
SRS          0.857731
TS%          0.857143
0-3ft_FG%    0.853448
2P%          0.842857
dtype: float64
PW      0.780488
NRtg    0.774194
MOV     0.771687
SRS     0.757143
W       0.746032
ORtg    0.557823
2P%     0.550725
Age     0.507463
FG      0.466667
FG%     0.444444
dtype: float64
            W        PW       SRS      NRtg       MOV      eFG%       FG%  \
11   0.918367  0.840909  0.855377  0.833333  0.837917  0.431818  0.444444   
40   0.888889  0.813953  0.849744  0.803371  0.803886  0.966667  0.803571   
66   0.906977  0.909091  0.920056  0.888889  0.900407  0.741573  0.859375   
94   1.000000  0.957447  0.885033  0.

In [115]:
# percentile feature importance

from itertools import combinations
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

champion_p = df_p[df_p['Champion'] == 1]
champion_p = champion_p.drop(columns=['L', 'PL', 'Champion', 'Levels', 'Year', 'Team'])

col_means = champion_p.mean()
sorted_col_means = col_means.sort_values(ascending=False)
print(sorted_col_means.head(10))

col_meds = champion_p.median()
sorted_col_meds = col_meds.sort_values(ascending=False)
print(sorted_col_meds.head(10))

col_mins = champion_p.min()
sorted_col_mins = col_mins.sort_values(ascending=False)
print(sorted_col_mins.head(10))

important_p = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%', '2P%', 'FG']

print(champion_p[important_p])

pairs1_p = list(combinations(important_p, 1))
pairs2_p = list(combinations(important_p, 2))
pairs3_p = list(combinations(important_p, 3))
pairs4_p = list(combinations(important_p, 4))
pairs5_p = list(combinations(important_p, 5))

for pair in pairs2_p:
    vif_data = pd.DataFrame()
    df_subset = df_p[list(pair)]  # Select only the columns in the pair

    vif_data["Feature"] = df_subset.columns
    vif_data["VIF"] = [variance_inflation_factor(df_subset.values, i) for i in range(df_subset.shape[1])]

    print(f"VIF for features: {pair}")
    print(vif_data)
    print("-" * 50)  # Separator for better readability


W       0.937931
eFG%    0.920690
SRS     0.919540
2P%     0.913793
MOV     0.912644
PW      0.911494
NRtg    0.910345
TS%     0.893103
FG%     0.877011
Age     0.837931
dtype: float64
eFG%         0.965517
W            0.931034
FG%          0.931034
SRS          0.931034
MOV          0.931034
2P%          0.931034
PW           0.931034
FG           0.896552
0-3ft_FG%    0.896552
TS%          0.896552
dtype: float64
W       0.793103
NRtg    0.758621
SRS     0.758621
MOV     0.758621
PW      0.758621
2P%     0.568966
PTS     0.517241
DRB     0.482759
eFG%    0.482759
Age     0.482759
dtype: float64
            W        PW       SRS      NRtg       MOV      eFG%       FG%  \
11   0.931034  0.827586  0.862069  0.810345  0.827586  0.482759  0.413793   
40   0.879310  0.758621  0.758621  0.758621  0.758621  0.931034  0.879310   
66   0.896552  0.931034  0.896552  0.896552  0.896552  0.844828  0.896552   
94   1.000000  0.965517  0.965517  0.965517  0.965517  1.000000  1.000000   
125  1.000

In [47]:
model1 = LogisticRegression(solver='lbfgs', max_iter=1000)

model2 = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000)

In [143]:
#min max data on model1 with 3 variables

results1_mm3 =pd.concat([multi_logistic_accuracy(df_mm, model1, pair) for pair in pairs3_mm])
results1_mm3 = results1_mm3.sort_values(by="Accuracy", ascending=False)
results1_mm3.head(10)

,Selected Features,Accuracy
0,"(W, MOV, eFG%)",0.5
0,"(W, SRS, eFG%)",0.5
0,"(PW, NRtg, eFG%)",0.5
0,"(SRS, NRtg, eFG%)",0.5
0,"(PW, SRS, eFG%)",0.5
0,"(SRS, MOV, eFG%)",0.5
0,"(NRtg, MOV, eFG%)",0.5
0,"(W, NRtg, eFG%)",0.5
0,"(PW, MOV, eFG%)",0.5
0,"(W, PW, eFG%)",0.5


In [141]:
#min max data on model1 with 2 variables

results1_mm2 =pd.concat([multi_logistic_accuracy(df_mm, model1, pair) for pair in pairs2_mm])
results1_mm2 = results1_mm2.sort_values(by="Accuracy", ascending=False)
results1_mm2.head(10)

,Selected Features,Accuracy
0,"(W, eFG%)",0.6
0,"(MOV, eFG%)",0.5
0,"(SRS, FG%)",0.5
0,"(SRS, eFG%)",0.5
0,"(W, FG%)",0.5
0,"(NRtg, eFG%)",0.5
0,"(PW, eFG%)",0.5
0,"(PW, FG%)",0.4
0,"(NRtg, FG%)",0.4
0,"(NRtg, TS%)",0.4


In [139]:
#min max data on model1 with 1 variables

results1_mm1 =pd.concat([multi_logistic_accuracy(df_mm, model1, pair) for pair in pairs1_mm])
results1_mm1 = results1_mm1.sort_values(by="Accuracy", ascending=False)
results1_mm1.head(10)

,Selected Features,Accuracy
0,"(FG%,)",0.5
0,"(eFG%,)",0.4
0,"(TS%,)",0.4
0,"(W,)",0.3
0,"(PW,)",0.3
0,"(SRS,)",0.3
0,"(NRtg,)",0.3
0,"(MOV,)",0.3


In [137]:
#min max data on model2 with 3 variables

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_mm3 =pd.concat([multi_logistic_accuracy(df_mm, model2, pair) for pair in pairs3_mm])
results2_mm3 = results2_mm3.sort_values(by="Accuracy", ascending=False)
results2_mm3.head(10)

,Selected Features,Accuracy
0,"(W, MOV, eFG%)",0.5
0,"(W, SRS, eFG%)",0.5
0,"(PW, NRtg, eFG%)",0.5
0,"(SRS, NRtg, eFG%)",0.5
0,"(PW, SRS, eFG%)",0.5
0,"(SRS, MOV, eFG%)",0.5
0,"(NRtg, MOV, eFG%)",0.5
0,"(W, NRtg, eFG%)",0.5
0,"(PW, MOV, eFG%)",0.5
0,"(W, PW, eFG%)",0.5


In [135]:
#min max data on model2 with 2 variables

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_mm2 =pd.concat([multi_logistic_accuracy(df_mm, model2, pair) for pair in pairs2_mm])
results2_mm2 = results2_mm2.sort_values(by="Accuracy", ascending=False)
results2_mm2.head(10)

,Selected Features,Accuracy
0,"(MOV, eFG%)",0.5
0,"(SRS, FG%)",0.5
0,"(SRS, eFG%)",0.5
0,"(W, eFG%)",0.5
0,"(W, FG%)",0.5
0,"(NRtg, eFG%)",0.5
0,"(PW, eFG%)",0.5
0,"(PW, FG%)",0.4
0,"(NRtg, FG%)",0.4
0,"(NRtg, TS%)",0.4


In [133]:
#min max data on model2 with 1 variables

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_mm1 =pd.concat([multi_logistic_accuracy(df_mm, model2, pair) for pair in pairs1_mm])
results2_mm1 = results2_mm1.sort_values(by="Accuracy", ascending=False)
results2_mm1.head(10)

,Selected Features,Accuracy
0,"(FG%,)",0.5
0,"(eFG%,)",0.4
0,"(TS%,)",0.4
0,"(W,)",0.3
0,"(PW,)",0.3
0,"(SRS,)",0.3
0,"(NRtg,)",0.3
0,"(MOV,)",0.3


In [117]:
#percentile data on model1 with 3 variables

results1_p3 =pd.concat([multi_logistic_accuracy(df_p, model1, pair) for pair in pairs3_p])
results1_p3 = results1_p3.sort_values(by="Accuracy", ascending=False)
results1_p3.head(10)

,Selected Features,Accuracy
0,"(SRS, eFG%, FG)",0.6
0,"(W, eFG%, FG)",0.6
0,"(PW, FG%, FG)",0.5
0,"(NRtg, FG%, FG)",0.5
0,"(W, FG%, FG)",0.5
0,"(W, 2P%, FG)",0.5
0,"(MOV, eFG%, FG)",0.5
0,"(PW, SRS, eFG%)",0.5
0,"(MOV, eFG%, TS%)",0.5
0,"(PW, NRtg, eFG%)",0.5


In [119]:
#percentile data on model1 with 2 variables

results1_p2 =pd.concat([multi_logistic_accuracy(df_p, model1, pair) for pair in pairs2_p])
results1_p2 = results1_p2.sort_values(by="Accuracy", ascending=False)
results1_p2.head(10)

,Selected Features,Accuracy
0,"(SRS, eFG%)",0.6
0,"(NRtg, eFG%)",0.5
0,"(SRS, FG%)",0.5
0,"(PW, eFG%)",0.5
0,"(eFG%, TS%)",0.5
0,"(eFG%, FG)",0.5
0,"(MOV, eFG%)",0.5
0,"(W, FG%)",0.5
0,"(W, eFG%)",0.5
0,"(FG%, FG)",0.5


In [121]:
#percentile data on model1 with 1 variables

results1_p1 =pd.concat([multi_logistic_accuracy(df_p, model1, pair) for pair in pairs1_p])
results1_p1 = results1_p1.sort_values(by="Accuracy", ascending=False)
results1_p1.head(10)

,Selected Features,Accuracy
0,"(FG%,)",0.5
0,"(eFG%,)",0.4
0,"(TS%,)",0.4
0,"(FG,)",0.4
0,"(W,)",0.3
0,"(PW,)",0.3
0,"(SRS,)",0.3
0,"(NRtg,)",0.3
0,"(MOV,)",0.3
0,"(2P%,)",0.2


In [123]:
# percentile data on model2 with 3 variables

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_p3 =pd.concat([multi_logistic_accuracy(df_p, model2, pair) for pair in pairs3_p])
results2_p3 = results2_p3.sort_values(by="Accuracy", ascending=False)
results2_p3.head(10)

,Selected Features,Accuracy
0,"(W, eFG%, FG)",0.6
0,"(SRS, eFG%, FG)",0.6
0,"(PW, eFG%, FG)",0.5
0,"(MOV, FG%, FG)",0.5
0,"(PW, SRS, eFG%)",0.5
0,"(MOV, eFG%, FG)",0.5
0,"(PW, NRtg, eFG%)",0.5
0,"(MOV, eFG%, TS%)",0.5
0,"(PW, MOV, eFG%)",0.5
0,"(PW, eFG%, TS%)",0.5


In [125]:
# percentile data on model2 with 2 variables

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_p2 =pd.concat([multi_logistic_accuracy(df_p, model2, pair) for pair in pairs2_p])
results2_p2 = results2_p2.sort_values(by="Accuracy", ascending=False)
results2_p2.head(10)

,Selected Features,Accuracy
0,"(W, eFG%)",0.6
0,"(MOV, eFG%)",0.6
0,"(SRS, eFG%)",0.6
0,"(FG%, FG)",0.5
0,"(W, FG%)",0.5
0,"(NRtg, eFG%)",0.5
0,"(eFG%, FG)",0.5
0,"(eFG%, TS%)",0.5
0,"(PW, eFG%)",0.5
0,"(SRS, FG%)",0.5


In [127]:
# percentile data on model2 with 1 variable

import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

results2_p1 =pd.concat([multi_logistic_accuracy(df_p, model2, pair) for pair in pairs1_p])
results2_p1 = results2_p1.sort_values(by="Accuracy", ascending=False)
results2_p1.head(10)

,Selected Features,Accuracy
0,"(FG%,)",0.5
0,"(eFG%,)",0.4
0,"(TS%,)",0.4
0,"(FG,)",0.4
0,"(W,)",0.3
0,"(PW,)",0.3
0,"(SRS,)",0.3
0,"(NRtg,)",0.3
0,"(MOV,)",0.3
0,"(2P%,)",0.2


In [145]:
# variables that get 60% accuracy

mm_model1_2_1_features = ['W', 'eFG%']

p_model1_2_1_features = ['SRS', 'eFG%']

p_model1_3_1_features = ['SRS', 'eFG%', 'FG']

p_model1_3_2_features = ['W', 'eFG%', 'FG']

p_model2_2_1_features = ['W', 'eFG%']

p_model2_2_2_features = ['MOV', 'eFG%']

p_model2_2_3_features = ['SRS', 'eFG%']

p_model2_3_1_features = ['W', 'eFG%', 'FG']

p_model2_3_2_features = ['SRS', 'eFG%', 'FG']

In [147]:
multi_logistic_top5(df_mm, model1, mm_model1_2_1_features)

  Feature     VIF
0       W  7.7102
1    eFG%  7.7102
  Feature       VIF
0       W  8.250266
1    eFG%  8.250266
  Feature       VIF
0       W  9.081993
1    eFG%  9.081993
  Feature       VIF
0       W  7.717541
1    eFG%  7.717541
  Feature      VIF
0       W  7.93086
1    eFG%  7.93086
  Feature       VIF
0       W  9.962458
1    eFG%  9.962458
  Feature       VIF
0       W  8.377401
1    eFG%  8.377401
  Feature       VIF
0       W  8.606272
1    eFG%  8.606272
  Feature       VIF
0       W  7.979711
1    eFG%  7.979711
  Feature       VIF
0       W  8.130166
1    eFG%  8.130166
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0747,Oklahoma City Thunder: 0.0707,Indiana Pacers: 0.0684,Denver Nuggets: 0.0617,Milwaukee Bucks: 0.0611
8,2022,Denver Nuggets,Denver Nuggets: 0.0827,Sacramento Kings: 0.0802,Golden State Warriors: 0.0744,Boston Celtics: 0.0727,Philadelphia 76ers: 0.0696
7,2021,Golden State Warriors,Golden State Warriors: 0.0885,Utah Jazz: 0.0883,Denver Nuggets: 0.0883,Phoenix Suns: 0.0876,Miami Heat: 0.0798
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.1009,Milwaukee Bucks: 0.0901,Phoenix Suns: 0.0895,Utah Jazz: 0.0885,Los Angeles Clippers: 0.0884
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1150,Utah Jazz: 0.1017,Miami Heat: 0.0969,Los Angeles Lakers: 0.0933,Dallas Mavericks: 0.0906
4,2018,Toronto Raptors,Golden State Warriors: 0.1237,Milwaukee Bucks: 0.0979,Toronto Raptors: 0.0852,Houston Rockets: 0.0794,Utah Jazz: 0.0694
3,2017,Golden State Warriors,Golden State Warriors: 0.1208,Houston Rockets: 0.0941,Toronto Raptors: 0.0689,Cleveland Cavaliers: 0.0662,New Orleans Pelicans: 0.0536
2,2016,Golden State Warriors,Golden State Warriors: 0.1029,Houston Rockets: 0.0689,Cleveland Cavaliers: 0.0648,San Antonio Spurs: 0.0542,Los Angeles Clippers: 0.0538
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.1039,San Antonio Spurs: 0.0638,Cleveland Cavaliers: 0.0532,Oklahoma City Thunder: 0.0510,Los Angeles Clippers: 0.0486
0,2014,Golden State Warriors,Golden State Warriors: 0.0865,Los Angeles Clippers: 0.0730,Atlanta Hawks: 0.0720,San Antonio Spurs: 0.0593,Cleveland Cavaliers: 0.0589


In [149]:
multi_logistic_top5(df_p, model1, p_model1_2_1_features)

  Feature       VIF
0     SRS  6.566913
1    eFG%  6.566913
  Feature       VIF
0     SRS  7.193042
1    eFG%  7.193042
  Feature     VIF
0     SRS  7.2069
1    eFG%  7.2069
  Feature       VIF
0     SRS  6.999133
1    eFG%  6.999133
  Feature       VIF
0     SRS  7.125978
1    eFG%  7.125978
  Feature       VIF
0     SRS  7.416846
1    eFG%  7.416846
  Feature       VIF
0     SRS  7.501679
1    eFG%  7.501679
  Feature       VIF
0     SRS  7.776179
1    eFG%  7.776179
  Feature       VIF
0     SRS  7.908657
1    eFG%  7.908657
  Feature       VIF
0     SRS  7.001353
1    eFG%  7.001353
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0861,Oklahoma City Thunder: 0.0802,Indiana Pacers: 0.0781,Denver Nuggets: 0.0669,Phoenix Suns: 0.0663
8,2022,Denver Nuggets,Denver Nuggets: 0.0841,Sacramento Kings: 0.0793,Boston Celtics: 0.0779,Golden State Warriors: 0.0730,Philadelphia 76ers: 0.0726
7,2021,Golden State Warriors,Utah Jazz: 0.0850,Golden State Warriors: 0.0821,Phoenix Suns: 0.0804,Denver Nuggets: 0.0759,Miami Heat: 0.0749
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.0835,Brooklyn Nets: 0.0832,Los Angeles Clippers: 0.0812,Phoenix Suns: 0.0806,Utah Jazz: 0.0783
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0921,Utah Jazz: 0.0796,Los Angeles Lakers: 0.0794,Dallas Mavericks: 0.0785,Miami Heat: 0.0783
4,2018,Toronto Raptors,Golden State Warriors: 0.0949,Milwaukee Bucks: 0.0931,Toronto Raptors: 0.0872,Houston Rockets: 0.0807,Utah Jazz: 0.0792
3,2017,Golden State Warriors,Golden State Warriors: 0.0946,Houston Rockets: 0.0942,Toronto Raptors: 0.0821,Philadelphia 76ers: 0.0703,Utah Jazz: 0.0647
2,2016,Golden State Warriors,Golden State Warriors: 0.0920,Houston Rockets: 0.0832,Cleveland Cavaliers: 0.0795,Los Angeles Clippers: 0.0787,Utah Jazz: 0.0659
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0902,San Antonio Spurs: 0.0860,Oklahoma City Thunder: 0.0787,Cleveland Cavaliers: 0.0774,Los Angeles Clippers: 0.0760
0,2014,Golden State Warriors,Golden State Warriors: 0.0794,Los Angeles Clippers: 0.0770,Atlanta Hawks: 0.0738,San Antonio Spurs: 0.0704,Cleveland Cavaliers: 0.0699


In [151]:
multi_logistic_top5(df_p, model1, p_model1_3_1_features)

  Feature       VIF
0     SRS  6.668918
1    eFG%  9.298105
2      FG  4.805148
  Feature       VIF
0     SRS  7.445954
1    eFG%  9.661789
2      FG  5.188046
  Feature       VIF
0     SRS  7.656538
1    eFG%  9.491794
2      FG  5.615828
  Feature       VIF
0     SRS  7.351701
1    eFG%  9.202642
2      FG  5.188120
  Feature       VIF
0     SRS  7.674278
1    eFG%  8.783290
2      FG  4.973306
  Feature       VIF
0     SRS  7.791549
1    eFG%  9.352512
2      FG  4.894679
  Feature       VIF
0     SRS  7.786323
1    eFG%  9.399810
2      FG  4.551379
  Feature       VIF
0     SRS  8.057876
1    eFG%  9.747417
2      FG  4.644169
  Feature        VIF
0     SRS   8.182404
1    eFG%  10.084523
2      FG   4.892282
  Feature       VIF
0     SRS  7.483801
1    eFG%  8.430267
2      FG  4.448781
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.1018,Indiana Pacers: 0.1014,Oklahoma City Thunder: 0.1003,Denver Nuggets: 0.0829,Milwaukee Bucks: 0.0691
8,2022,Denver Nuggets,Denver Nuggets: 0.1010,Sacramento Kings: 0.0959,Golden State Warriors: 0.0834,Boston Celtics: 0.0736,Milwaukee Bucks: 0.0645
7,2021,Golden State Warriors,Phoenix Suns: 0.1171,Milwaukee Bucks: 0.0859,Denver Nuggets: 0.0849,Utah Jazz: 0.0677,Boston Celtics: 0.0612
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.1145,Phoenix Suns: 0.1040,Brooklyn Nets: 0.0961,Denver Nuggets: 0.0947,Los Angeles Clippers: 0.0756
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1217,Los Angeles Lakers: 0.0971,Dallas Mavericks: 0.0807,New Orleans Pelicans: 0.0700,Los Angeles Clippers: 0.0681
4,2018,Toronto Raptors,Golden State Warriors: 0.1307,Milwaukee Bucks: 0.1184,Toronto Raptors: 0.0869,Boston Celtics: 0.0646,San Antonio Spurs: 0.0629
3,2017,Golden State Warriors,Golden State Warriors: 0.1261,Toronto Raptors: 0.1038,New Orleans Pelicans: 0.0828,Philadelphia 76ers: 0.0824,Denver Nuggets: 0.0633
2,2016,Golden State Warriors,Golden State Warriors: 0.1159,Houston Rockets: 0.0971,Cleveland Cavaliers: 0.0878,Washington Wizards: 0.0800,Los Angeles Clippers: 0.0759
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.1128,San Antonio Spurs: 0.1025,Oklahoma City Thunder: 0.0968,Cleveland Cavaliers: 0.0817,Atlanta Hawks: 0.0667
0,2014,Golden State Warriors,Golden State Warriors: 0.1004,Los Angeles Clippers: 0.0929,San Antonio Spurs: 0.0839,Dallas Mavericks: 0.0750,Atlanta Hawks: 0.0748


In [153]:
multi_logistic_top5(df_p, model1, p_model1_3_2_features)

  Feature       VIF
0       W  6.063956
1    eFG%  9.366013
2      FG  4.743582
  Feature       VIF
0       W  6.827360
1    eFG%  9.759060
2      FG  5.079293
  Feature       VIF
0       W  6.787618
1    eFG%  9.132842
2      FG  5.519639
  Feature       VIF
0       W  6.733322
1    eFG%  8.895595
2      FG  5.136954
  Feature       VIF
0       W  7.406294
1    eFG%  8.683857
2      FG  4.938263
  Feature       VIF
0       W  7.600329
1    eFG%  8.948818
2      FG  4.959327
  Feature       VIF
0       W  7.217473
1    eFG%  8.646736
2      FG  4.612075
  Feature       VIF
0       W  7.647025
1    eFG%  9.197475
2      FG  4.685502
  Feature       VIF
0       W  7.699093
1    eFG%  9.307295
2      FG  4.972451
  Feature       VIF
0       W  7.072228
1    eFG%  8.089511
2      FG  4.450623
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0997,Oklahoma City Thunder: 0.0990,Indiana Pacers: 0.0946,Denver Nuggets: 0.0825,Milwaukee Bucks: 0.0778
8,2022,Denver Nuggets,Denver Nuggets: 0.1006,Sacramento Kings: 0.0979,Golden State Warriors: 0.0807,Boston Celtics: 0.0713,Milwaukee Bucks: 0.0638
7,2021,Golden State Warriors,Phoenix Suns: 0.1136,Denver Nuggets: 0.0919,Milwaukee Bucks: 0.0871,Utah Jazz: 0.0656,Charlotte Hornets: 0.0599
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.1144,Phoenix Suns: 0.1048,Brooklyn Nets: 0.0992,Denver Nuggets: 0.0975,Los Angeles Clippers: 0.0746
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1210,Los Angeles Lakers: 0.0974,Dallas Mavericks: 0.0691,Denver Nuggets: 0.0691,Los Angeles Clippers: 0.0663
4,2018,Toronto Raptors,Golden State Warriors: 0.1296,Milwaukee Bucks: 0.1167,Toronto Raptors: 0.0869,San Antonio Spurs: 0.0650,Boston Celtics: 0.0635
3,2017,Golden State Warriors,Golden State Warriors: 0.1275,Toronto Raptors: 0.1043,New Orleans Pelicans: 0.0956,Philadelphia 76ers: 0.0841,Cleveland Cavaliers: 0.0800
2,2016,Golden State Warriors,Golden State Warriors: 0.1167,Houston Rockets: 0.0984,Cleveland Cavaliers: 0.0908,Washington Wizards: 0.0825,Los Angeles Clippers: 0.0739
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.1160,San Antonio Spurs: 0.1055,Oklahoma City Thunder: 0.0960,Cleveland Cavaliers: 0.0855,Los Angeles Clippers: 0.0666
0,2014,Golden State Warriors,Golden State Warriors: 0.1046,Los Angeles Clippers: 0.0951,San Antonio Spurs: 0.0837,Atlanta Hawks: 0.0793,Dallas Mavericks: 0.0752


In [155]:
import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

multi_logistic_top5(df_p, model2, p_model2_2_1_features)

  Feature       VIF
0       W  6.048703
1    eFG%  6.048703
  Feature       VIF
0       W  6.736676
1    eFG%  6.736676
  Feature       VIF
0       W  6.500348
1    eFG%  6.500348
  Feature       VIF
0       W  6.474259
1    eFG%  6.474259
  Feature       VIF
0       W  6.925942
1    eFG%  6.925942
  Feature       VIF
0       W  7.140512
1    eFG%  7.140512
  Feature       VIF
0       W  6.862114
1    eFG%  6.862114
  Feature       VIF
0       W  7.314591
1    eFG%  7.314591
  Feature       VIF
0       W  7.321538
1    eFG%  7.321538
  Feature       VIF
0       W  6.613574
1    eFG%  6.613574
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0746,Oklahoma City Thunder: 0.0702,Milwaukee Bucks: 0.0646,Indiana Pacers: 0.0643,Denver Nuggets: 0.0609
8,2022,Denver Nuggets,Denver Nuggets: 0.0735,Sacramento Kings: 0.0706,Boston Celtics: 0.0676,Philadelphia 76ers: 0.0638,Golden State Warriors: 0.0624
7,2021,Golden State Warriors,Golden State Warriors: 0.0714,Denver Nuggets: 0.0710,Utah Jazz: 0.0706,Phoenix Suns: 0.0694,Miami Heat: 0.0672
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.0760,Milwaukee Bucks: 0.0721,Phoenix Suns: 0.0719,Los Angeles Clippers: 0.0702,Utah Jazz: 0.0693
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0799,Utah Jazz: 0.0712,Los Angeles Lakers: 0.0703,Miami Heat: 0.0686,Toronto Raptors: 0.0645
4,2018,Toronto Raptors,Golden State Warriors: 0.0809,Milwaukee Bucks: 0.0795,Toronto Raptors: 0.0766,Houston Rockets: 0.0717,Utah Jazz: 0.0662
3,2017,Golden State Warriors,Golden State Warriors: 0.0830,Houston Rockets: 0.0818,Cleveland Cavaliers: 0.0745,Toronto Raptors: 0.0733,New Orleans Pelicans: 0.0659
2,2016,Golden State Warriors,Golden State Warriors: 0.0804,Houston Rockets: 0.0745,Cleveland Cavaliers: 0.0733,Los Angeles Clippers: 0.0687,Boston Celtics: 0.0608
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0808,San Antonio Spurs: 0.0777,Cleveland Cavaliers: 0.0722,Oklahoma City Thunder: 0.0703,Los Angeles Clippers: 0.0691
0,2014,Golden State Warriors,Golden State Warriors: 0.0730,Los Angeles Clippers: 0.0699,Atlanta Hawks: 0.0694,San Antonio Spurs: 0.0635,Cleveland Cavaliers: 0.0632


In [157]:
import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

multi_logistic_top5(df_p, model2, p_model2_2_2_features)

  Feature     VIF
0     MOV  6.4571
1    eFG%  6.4571
  Feature       VIF
0     MOV  6.948479
1    eFG%  6.948479
  Feature       VIF
0     MOV  6.981837
1    eFG%  6.981837
  Feature       VIF
0     MOV  6.931575
1    eFG%  6.931575
  Feature       VIF
0     MOV  7.018316
1    eFG%  7.018316
  Feature       VIF
0     MOV  7.372382
1    eFG%  7.372382
  Feature       VIF
0     MOV  7.176522
1    eFG%  7.176522
  Feature       VIF
0     MOV  7.394214
1    eFG%  7.394214
  Feature       VIF
0     MOV  7.482804
1    eFG%  7.482804
  Feature       VIF
0     MOV  6.714873
1    eFG%  6.714873
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0748,Indiana Pacers: 0.0705,Oklahoma City Thunder: 0.0703,Milwaukee Bucks: 0.0609,Phoenix Suns: 0.0603
8,2022,Denver Nuggets,Denver Nuggets: 0.0731,Sacramento Kings: 0.0696,Boston Celtics: 0.0683,Philadelphia 76ers: 0.0641,Golden State Warriors: 0.0637
7,2021,Golden State Warriors,Utah Jazz: 0.0745,Golden State Warriors: 0.0714,Phoenix Suns: 0.0712,Denver Nuggets: 0.0691,Miami Heat: 0.0664
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.0735,Brooklyn Nets: 0.0730,Los Angeles Clippers: 0.0710,Phoenix Suns: 0.0701,Utah Jazz: 0.0685
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0804,Utah Jazz: 0.0707,Dallas Mavericks: 0.0699,Miami Heat: 0.0698,Los Angeles Lakers: 0.0687
4,2018,Toronto Raptors,Golden State Warriors: 0.0824,Milwaukee Bucks: 0.0809,Toronto Raptors: 0.0767,Houston Rockets: 0.0721,Utah Jazz: 0.0709
3,2017,Golden State Warriors,Golden State Warriors: 0.0828,Houston Rockets: 0.0821,Toronto Raptors: 0.0734,Philadelphia 76ers: 0.0662,Utah Jazz: 0.0590
2,2016,Golden State Warriors,Golden State Warriors: 0.0803,Houston Rockets: 0.0741,Cleveland Cavaliers: 0.0720,Los Angeles Clippers: 0.0709,San Antonio Spurs: 0.0602
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0791,San Antonio Spurs: 0.0761,Oklahoma City Thunder: 0.0707,Cleveland Cavaliers: 0.0700,Los Angeles Clippers: 0.0681
0,2014,Golden State Warriors,Golden State Warriors: 0.0697,Los Angeles Clippers: 0.0679,Atlanta Hawks: 0.0656,Cleveland Cavaliers: 0.0635,San Antonio Spurs: 0.0630


In [159]:
import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

multi_logistic_top5(df_p, model2, p_model2_2_3_features)

  Feature       VIF
0     SRS  6.566913
1    eFG%  6.566913
  Feature       VIF
0     SRS  7.193042
1    eFG%  7.193042
  Feature     VIF
0     SRS  7.2069
1    eFG%  7.2069
  Feature       VIF
0     SRS  6.999133
1    eFG%  6.999133
  Feature       VIF
0     SRS  7.125978
1    eFG%  7.125978
  Feature       VIF
0     SRS  7.416846
1    eFG%  7.416846
  Feature       VIF
0     SRS  7.501679
1    eFG%  7.501679
  Feature       VIF
0     SRS  7.776179
1    eFG%  7.776179
  Feature       VIF
0     SRS  7.908657
1    eFG%  7.908657
  Feature       VIF
0     SRS  7.001353
1    eFG%  7.001353
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0758,Oklahoma City Thunder: 0.0712,Indiana Pacers: 0.0695,Denver Nuggets: 0.0609,Phoenix Suns: 0.0604
8,2022,Denver Nuggets,Denver Nuggets: 0.0733,Sacramento Kings: 0.0695,Boston Celtics: 0.0694,Philadelphia 76ers: 0.0649,Golden State Warriors: 0.0647
7,2021,Golden State Warriors,Utah Jazz: 0.0749,Golden State Warriors: 0.0722,Phoenix Suns: 0.0713,Denver Nuggets: 0.0667,Miami Heat: 0.0665
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.0736,Brooklyn Nets: 0.0730,Los Angeles Clippers: 0.0719,Phoenix Suns: 0.0713,Utah Jazz: 0.0696
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0806,Utah Jazz: 0.0706,Los Angeles Lakers: 0.0705,Dallas Mavericks: 0.0699,Miami Heat: 0.0696
4,2018,Toronto Raptors,Golden State Warriors: 0.0827,Milwaukee Bucks: 0.0813,Toronto Raptors: 0.0768,Houston Rockets: 0.0720,Utah Jazz: 0.0710
3,2017,Golden State Warriors,Golden State Warriors: 0.0830,Houston Rockets: 0.0825,Toronto Raptors: 0.0737,Philadelphia 76ers: 0.0649,Utah Jazz: 0.0605
2,2016,Golden State Warriors,Golden State Warriors: 0.0808,Houston Rockets: 0.0743,Cleveland Cavaliers: 0.0717,Los Angeles Clippers: 0.0710,Utah Jazz: 0.0611
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0796,San Antonio Spurs: 0.0765,Oklahoma City Thunder: 0.0710,Cleveland Cavaliers: 0.0701,Los Angeles Clippers: 0.0691
0,2014,Golden State Warriors,Golden State Warriors: 0.0705,Los Angeles Clippers: 0.0686,Atlanta Hawks: 0.0660,San Antonio Spurs: 0.0635,Cleveland Cavaliers: 0.0629


In [161]:
import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

multi_logistic_top5(df_p, model2, p_model2_3_1_features)

  Feature       VIF
0       W  6.063956
1    eFG%  9.366013
2      FG  4.743582
  Feature       VIF
0       W  6.827360
1    eFG%  9.759060
2      FG  5.079293
  Feature       VIF
0       W  6.787618
1    eFG%  9.132842
2      FG  5.519639
  Feature       VIF
0       W  6.733322
1    eFG%  8.895595
2      FG  5.136954
  Feature       VIF
0       W  7.406294
1    eFG%  8.683857
2      FG  4.938263
  Feature       VIF
0       W  7.600329
1    eFG%  8.948818
2      FG  4.959327
  Feature       VIF
0       W  7.217473
1    eFG%  8.646736
2      FG  4.612075
  Feature       VIF
0       W  7.647025
1    eFG%  9.197475
2      FG  4.685502
  Feature       VIF
0       W  7.699093
1    eFG%  9.307295
2      FG  4.972451
  Feature       VIF
0       W  7.072228
1    eFG%  8.089511
2      FG  4.450623
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0860,Oklahoma City Thunder: 0.0850,Indiana Pacers: 0.0817,Denver Nuggets: 0.0731,Milwaukee Bucks: 0.0693
8,2022,Denver Nuggets,Denver Nuggets: 0.0857,Sacramento Kings: 0.0830,Golden State Warriors: 0.0705,Boston Celtics: 0.0646,Milwaukee Bucks: 0.0586
7,2021,Golden State Warriors,Phoenix Suns: 0.0967,Denver Nuggets: 0.0794,Milwaukee Bucks: 0.0762,Utah Jazz: 0.0597,Charlotte Hornets: 0.0576
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.0965,Phoenix Suns: 0.0902,Brooklyn Nets: 0.0862,Denver Nuggets: 0.0840,Los Angeles Clippers: 0.0669
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1024,Los Angeles Lakers: 0.0849,Denver Nuggets: 0.0640,Dallas Mavericks: 0.0636,Los Angeles Clippers: 0.0616
4,2018,Toronto Raptors,Golden State Warriors: 0.1097,Milwaukee Bucks: 0.1001,Toronto Raptors: 0.0779,San Antonio Spurs: 0.0621,Boston Celtics: 0.0602
3,2017,Golden State Warriors,Golden State Warriors: 0.1095,Toronto Raptors: 0.0926,New Orleans Pelicans: 0.0879,Philadelphia 76ers: 0.0776,Cleveland Cavaliers: 0.0735
2,2016,Golden State Warriors,Golden State Warriors: 0.0997,Houston Rockets: 0.0868,Cleveland Cavaliers: 0.0814,Washington Wizards: 0.0762,Los Angeles Clippers: 0.0679
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0994,San Antonio Spurs: 0.0919,Oklahoma City Thunder: 0.0861,Cleveland Cavaliers: 0.0766,Los Angeles Clippers: 0.0615
0,2014,Golden State Warriors,Golden State Warriors: 0.0903,Los Angeles Clippers: 0.0834,San Antonio Spurs: 0.0749,Atlanta Hawks: 0.0710,Dallas Mavericks: 0.0684


In [163]:
import warnings

# Mute the specific FutureWarning for 'multi_class' deprecation in LogisticRegression
warnings.filterwarnings("ignore", category=FutureWarning, 
                        message=".*'multi_class' was deprecated.*")

multi_logistic_top5(df_p, model2, p_model2_3_2_features)

  Feature       VIF
0     SRS  6.668918
1    eFG%  9.298105
2      FG  4.805148
  Feature       VIF
0     SRS  7.445954
1    eFG%  9.661789
2      FG  5.188046
  Feature       VIF
0     SRS  7.656538
1    eFG%  9.491794
2      FG  5.615828
  Feature       VIF
0     SRS  7.351701
1    eFG%  9.202642
2      FG  5.188120
  Feature       VIF
0     SRS  7.674278
1    eFG%  8.783290
2      FG  4.973306
  Feature       VIF
0     SRS  7.791549
1    eFG%  9.352512
2      FG  4.894679
  Feature       VIF
0     SRS  7.786323
1    eFG%  9.399810
2      FG  4.551379
  Feature       VIF
0     SRS  8.057876
1    eFG%  9.747417
2      FG  4.644169
  Feature        VIF
0     SRS   8.182404
1    eFG%  10.084523
2      FG   4.892282
  Feature       VIF
0     SRS  7.483801
1    eFG%  8.430267
2      FG  4.448781
Model Accuracy: 60.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0872,Indiana Pacers: 0.0867,Oklahoma City Thunder: 0.0858,Denver Nuggets: 0.0731,Milwaukee Bucks: 0.0634
8,2022,Denver Nuggets,Denver Nuggets: 0.0856,Sacramento Kings: 0.0817,Golden State Warriors: 0.0728,Boston Celtics: 0.0665,Milwaukee Bucks: 0.0590
7,2021,Golden State Warriors,Phoenix Suns: 0.0988,Milwaukee Bucks: 0.0757,Denver Nuggets: 0.0748,Utah Jazz: 0.0625,Charlotte Hornets: 0.0580
6,2020,Milwaukee Bucks,Milwaukee Bucks: 0.0975,Phoenix Suns: 0.0895,Brooklyn Nets: 0.0835,Denver Nuggets: 0.0822,Los Angeles Clippers: 0.0680
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.1029,Los Angeles Lakers: 0.0849,Dallas Mavericks: 0.0726,New Orleans Pelicans: 0.0660,Los Angeles Clippers: 0.0630
4,2018,Toronto Raptors,Golden State Warriors: 0.1106,Milwaukee Bucks: 0.1015,Toronto Raptors: 0.0781,Boston Celtics: 0.0612,San Antonio Spurs: 0.0607
3,2017,Golden State Warriors,Golden State Warriors: 0.1089,Toronto Raptors: 0.0925,New Orleans Pelicans: 0.0788,Philadelphia 76ers: 0.0765,Denver Nuggets: 0.0614
2,2016,Golden State Warriors,Golden State Warriors: 0.0996,Houston Rockets: 0.0861,Cleveland Cavaliers: 0.0794,Washington Wizards: 0.0746,Los Angeles Clippers: 0.0694
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0978,San Antonio Spurs: 0.0902,Oklahoma City Thunder: 0.0865,Cleveland Cavaliers: 0.0742,Atlanta Hawks: 0.0623
0,2014,Golden State Warriors,Golden State Warriors: 0.0873,Los Angeles Clippers: 0.0816,San Antonio Spurs: 0.0747,Dallas Mavericks: 0.0678,Atlanta Hawks: 0.0674
